# Convolutional Nets 
La entrada de las capas convolucionales esta dada por el _(heigth, width, chanels)_ de la imagen, la salida igual.
Las redes convolucionales aprender patrones locales mientras que las de capa _Dense_ globalizan el conocimiento de toda la entrada como puede ser una imagen. Es decir las convolucionales aprenden cosas como orillas, ciertos recuadros y los generalizan después.

**Las redes convolucionales operan sobre tensores 3D también llamados _feature maps_, cuentan con 2 espacios dimensionales heigth & width, y _depth axis_ (también llamado _Channel_)**

``` Nota: Channels son los canales de gama de color, por ejemplo en una imagen con RGB el channel es 3. En una imagen a blanco y negro el canal es de 1 (la tonalidad de gris) ```

A convolution works by sliding, for instance some windows of size 3 × 3 or 5 × 5 over the 3D input
feature map, stopping at every possible location, and extracting the 3D patch of surrounding
features (shape (window_height, window_width, input_depth)). Each
such 3D patch is then transformed (via a tensor product with the same learned weight
matrix, **called the convolution kernel**) into a 1D vector of shape (output_depth,). All of
these vectors are then spatially reassembled into a 3D output map of shape (height,
width, output_depth). Every spatial location in the output feature map corresponds
to the same location in the input feature map


In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras import models

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

"""
model = models.Sequential()

model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)
    
test_loss, test_acc = model.evaluate(test_images, test_labels)
print test_acc
"""


Using TensorFlow backend.


"\nmodel = models.Sequential()\n\nmodel.compile(optimizer='rmsprop',\n                loss='categorical_crossentropy',\n                metrics=['accuracy'])\nmodel.fit(train_images, train_labels, epochs=5, batch_size=64)\n    \ntest_loss, test_acc = model.evaluate(test_images, test_labels)\nprint test_acc\n"

## Max Pooling
Es una manera de hacer **Downsampling** al input de datos. Consiste en extraer ventanas del feature map de input y devolver el mayor valor del channel.

In [2]:
# Extract small dataset from kagle Dogs vs Cats public dataset downloaded

import os, shutil

original_dataset_dir = '/home/rick/Downloads/dogs-vs-cats/train'
base_dir = '/home/rick/Desktop/machine-learning/small_cats_dogs'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dogs')

os.mkdir(train_dogs_dir)
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

os.mkdir(validation_dogs_dir)
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [ ]:
# We now create the model

from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

from keras import optimizers
model.compile(loss='binary_crossentropy',
                optimizer=optimizers.RMSprop(lr=1e-4),
                metrics=['acc'])





## Data Preprocesing <Pagina 149>
Ahora debemos completar una serie de pasos para poder introducir nuestros datos al modelo.
    
    1 Read the picture files.
    2 Decode the JPEG content to RGB grids of pixels.
    3 Convert these into floating-point tensors.
    4 Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know,
    neural networks prefer to deal with small input values).

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150)
                                                    batch_size=20,
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='binary')